In [1]:
import pandas as pd
from datetime import datetime, timedelta, date

In [2]:
data_location = 'C:/Users/kaley/OneDrive/Documents/Coding Stuff/Data Science/Mental Health Project/Sample Data/'
mood_data_location = data_location+'FinchExport_2025-04-12/Mood.json'

In [3]:
mood_series = pd.read_json(mood_data_location, typ='series')
mood_data = pd.json_normalize(mood_series.data)
mood_data['value'] = pd.to_numeric(mood_data.value)

In [4]:
mood_data.head()

,dt,mood_type,value,updated_time
0,"Sat, 6 May 2023 14:00:58",feeling,4,"Sat, 6 May 2023 14:00:58"
1,"Sun, 11 Feb 2024 04:06:37",satisfaction,4,"Sun, 11 Feb 2024 04:06:37"
2,"Wed, 2 Nov 2022 09:04:11",motivation,3,"Wed, 2 Nov 2022 09:04:11"
3,"Wed, 12 Jul 2023 19:02:37",feeling,3,"Wed, 12 Jul 2023 19:02:37"
4,"Sat, 1 Mar 2025 13:39:47",motivation,2,"Sat, 1 Mar 2025 13:39:47"


In [72]:
#mood_data.to_csv('mood_data.csv',index=False)

In [5]:
# note: a pandas update didn't like the non-zero-padded 'day' of the Finch data
# so I'm using datetime.strptime instead of pd.to_datetime for now
mood_data['Record_Datetime'] = mood_data.dt.map(lambda x: datetime.strptime(x, "%a, %d %b %Y %H:%M:%S"))
mood_data['Updated_Datetime'] = mood_data.updated_time.map(lambda x: datetime.strptime(x, "%a, %d %b %Y %H:%M:%S"))
mood_data = mood_data.drop(['dt','updated_time'],axis=1)

In [6]:
mood_data.head()

,mood_type,value,Record_Datetime,Updated_Datetime
0,feeling,4,2023-05-06 14:00:58,2023-05-06 14:00:58
1,satisfaction,4,2024-02-11 04:06:37,2024-02-11 04:06:37
2,motivation,3,2022-11-02 09:04:11,2022-11-02 09:04:11
3,feeling,3,2023-07-12 19:02:37,2023-07-12 19:02:37
4,motivation,2,2025-03-01 13:39:47,2025-03-01 13:39:47


In [7]:
mood_data['time_difference'] = mood_data['Updated_Datetime'] - mood_data['Record_Datetime']
mood_data.head()

,mood_type,value,Record_Datetime,Updated_Datetime,time_difference
0,feeling,4,2023-05-06 14:00:58,2023-05-06 14:00:58,0 days
1,satisfaction,4,2024-02-11 04:06:37,2024-02-11 04:06:37,0 days
2,motivation,3,2022-11-02 09:04:11,2022-11-02 09:04:11,0 days
3,feeling,3,2023-07-12 19:02:37,2023-07-12 19:02:37,0 days
4,motivation,2,2025-03-01 13:39:47,2025-03-01 13:39:47,0 days


In [8]:
mood_data['time_difference'].max()

Timedelta('0 days 00:00:00')

In [9]:
# not very helpful so dropping both columns
mood_data = mood_data.drop(['time_difference','Updated_Datetime'],axis=1)

In [10]:
mood_data['Record_Date'] = mood_data['Record_Datetime'].dt.date

In [11]:
mood_data.sort_values('Record_Datetime', ascending=False).head(10)

,mood_type,value,Record_Datetime,Record_Date
706,feeling,4,2025-04-12 16:20:05,2025-04-12
2795,feeling,4,2025-04-12 15:13:29,2025-04-12
3950,feeling,4,2025-04-12 12:29:14,2025-04-12
3096,feeling,3,2025-04-12 10:14:01,2025-04-12
1626,motivation,3,2025-04-12 09:12:00,2025-04-12
2459,satisfaction,4,2025-04-12 05:15:46,2025-04-12
624,feeling,3,2025-04-11 20:16:09,2025-04-11
925,feeling,4,2025-04-11 16:44:51,2025-04-11
2726,feeling,4,2025-04-11 15:03:02,2025-04-11
3278,motivation,4,2025-04-11 12:18:07,2025-04-11


In [12]:
satisfaction_data = mood_data.query("mood_type == 'satisfaction'")
satisfaction_data.head()

,mood_type,value,Record_Datetime,Record_Date
1,satisfaction,4,2024-02-11 04:06:37,2024-02-11
7,satisfaction,1,2022-11-02 22:06:19,2022-11-02
11,satisfaction,3,2024-11-03 01:17:46,2024-11-03
14,satisfaction,4,2022-12-14 03:26:55,2022-12-14
16,satisfaction,4,2023-05-20 00:25:57,2023-05-20


In [13]:
mood_counts = mood_data.groupby(['Record_Date','mood_type'])\
    .count().reset_index().drop(['Record_Datetime'], axis=1)\
    .rename(columns={'value':'count'})

In [14]:
mood_counts.head(5)

,Record_Date,mood_type,count
0,2022-10-26,feeling,6
1,2022-10-26,motivation,1
2,2022-10-26,satisfaction,1
3,2022-10-27,feeling,6
4,2022-10-27,motivation,1


In [15]:
# duplicates seem only only mood type satisfaction (makes sense since it is asked at the end of the day, 
#    motivation only asked at begining of day)
satisfaction_duplicates = mood_counts.query("mood_type == 'satisfaction'").query("count > 1")
satisfaction_non_duplicates = mood_counts.query("mood_type == 'satisfaction'").query("count == 1")

In [16]:
satisfaction_duplicates.head()

,Record_Date,mood_type,count
10,2022-10-29,satisfaction,2
18,2022-11-01,satisfaction,2
29,2022-11-05,satisfaction,2
57,2022-11-15,satisfaction,2
219,2023-01-14,satisfaction,2


In [17]:
satisfaction_duplicates = satisfaction_duplicates.set_index(["Record_Date","mood_type"])
sat_w_duplicates = satisfaction_duplicates.join(mood_data.set_index(["Record_Date","mood_type"]), lsuffix="_L", rsuffix="_R")
#
satisfaction_non_duplicates = satisfaction_non_duplicates.set_index(["Record_Date","mood_type"])
sat_wout_duplicates = satisfaction_non_duplicates.join(mood_data.set_index(["Record_Date","mood_type"]), lsuffix="_L", rsuffix="_R")

In [18]:
sat_w_duplicates.head()

count  value     Record_Datetime
Record_Date mood_type                                     
2022-10-29  satisfaction      2      3 2022-10-29 22:44:40
            satisfaction      2      4 2022-10-29 02:09:04
2022-11-01  satisfaction      2      4 2022-11-01 00:06:00
            satisfaction      2      3 2022-11-01 22:09:07
2022-11-05  satisfaction      2      3 2022-11-05 23:34:22

In [19]:
sat_w_duplicates = sat_w_duplicates.reset_index().drop('count',axis=1)
sat_w_duplicates['Record_Time'] = sat_w_duplicates['Record_Datetime'].dt.time
#
sat_wout_duplicates = sat_wout_duplicates.reset_index().drop('count',axis=1)
sat_wout_duplicates['Record_Time'] = sat_wout_duplicates['Record_Datetime'].dt.time


In [20]:
len(sat_w_duplicates)

230

In [21]:
sat_w_duplicates.head()

,Record_Date,mood_type,value,Record_Datetime,Record_Time
0,2022-10-29,satisfaction,3,2022-10-29 22:44:40,22:44:40
1,2022-10-29,satisfaction,4,2022-10-29 02:09:04,02:09:04
2,2022-11-01,satisfaction,4,2022-11-01 00:06:00,00:06:00
3,2022-11-01,satisfaction,3,2022-11-01 22:09:07,22:09:07
4,2022-11-05,satisfaction,3,2022-11-05 23:34:22,23:34:22


In [22]:
# my logic is that the duplicate date with the min time is the record that I ranked my satisfaction in the early hours of the next day
# so taking only the records that are duplicates and have the min time, I change the date to the date before
sat_w_duplicates['Modified_Date'] = False
modified_dates = sat_w_duplicates[['Record_Date','Record_Time','Modified_Date']]

mod_max_times = modified_dates.groupby('Record_Date').max().reset_index()

modified_dates = modified_dates.groupby('Record_Date').min().reset_index()

In [23]:
print(len(modified_dates))
modified_dates.head()

115


,Record_Date,Record_Time,Modified_Date
0,2022-10-29,02:09:04,False
1,2022-11-01,00:06:00,False
2,2022-11-05,01:48:01,False
3,2022-11-15,00:00:41,False
4,2023-01-14,00:41:50,False


In [24]:
modified_dates = modified_dates.set_index(['Record_Date','Record_Time']).join(sat_w_duplicates.set_index(['Record_Date','Record_Time']), rsuffix="_R")

In [25]:
print(len(modified_dates))
modified_dates.head()

115


,,Modified_Date,mood_type,value,Record_Datetime,Modified_Date_R
Record_Date,Record_Time,,,,,
2022-10-29,02:09:04,False,satisfaction,4,2022-10-29 02:09:04,False
2022-11-01,00:06:00,False,satisfaction,4,2022-11-01 00:06:00,False
2022-11-05,01:48:01,False,satisfaction,2,2022-11-05 01:48:01,False
2022-11-15,00:00:41,False,satisfaction,4,2022-11-15 00:00:41,False
2023-01-14,00:41:50,False,satisfaction,3,2023-01-14 00:41:50,False


In [26]:
modified_dates = modified_dates.reset_index()[['Record_Date','Record_Time','Modified_Date','mood_type','value']]

In [27]:
one_day = timedelta(days=1)
modified_dates['Record_Date_Mod'] = modified_dates['Record_Date'].map(lambda x: x - one_day)
modified_dates['Modified_Date'] = True

In [28]:
modified_dates.head()

,Record_Date,Record_Time,Modified_Date,mood_type,value,Record_Date_Mod
0,2022-10-29,02:09:04,True,satisfaction,4,2022-10-28
1,2022-11-01,00:06:00,True,satisfaction,4,2022-10-31
2,2022-11-05,01:48:01,True,satisfaction,2,2022-11-04
3,2022-11-15,00:00:41,True,satisfaction,4,2022-11-14
4,2023-01-14,00:41:50,True,satisfaction,3,2023-01-13


In [29]:
modified_dates_mins = modified_dates[['Record_Date_Mod','Record_Time','value','Modified_Date']]
modified_dates_mins = modified_dates_mins.rename(columns={'Record_Date_Mod':'Record_Date'})
modified_dates_mins.head()

,Record_Date,Record_Time,value,Modified_Date
0,2022-10-28,02:09:04,4,True
1,2022-10-31,00:06:00,4,True
2,2022-11-04,01:48:01,2,True
3,2022-11-14,00:00:41,4,True
4,2023-01-13,00:41:50,3,True


In [30]:
# duplicates that don't need modified dates
mod_max_times.head()

,Record_Date,Record_Time,Modified_Date
0,2022-10-29,22:44:40,False
1,2022-11-01,22:09:07,False
2,2022-11-05,23:34:22,False
3,2022-11-15,22:52:41,False
4,2023-01-14,22:43:54,False


In [31]:
modified_dates_maxes = mod_max_times.set_index(['Record_Date','Record_Time']).join(sat_w_duplicates.set_index(['Record_Date','Record_Time']), rsuffix="_R")

In [32]:
modified_dates_maxes.head()

,,Modified_Date,mood_type,value,Record_Datetime,Modified_Date_R
Record_Date,Record_Time,,,,,
2022-10-29,22:44:40,False,satisfaction,3,2022-10-29 22:44:40,False
2022-11-01,22:09:07,False,satisfaction,3,2022-11-01 22:09:07,False
2022-11-05,23:34:22,False,satisfaction,3,2022-11-05 23:34:22,False
2022-11-15,22:52:41,False,satisfaction,5,2022-11-15 22:52:41,False
2023-01-14,22:43:54,False,satisfaction,5,2023-01-14 22:43:54,False


In [33]:
modified_dates_maxes = modified_dates_maxes.reset_index()[['Record_Date','Record_Time','Modified_Date','value']]

In [34]:
modified_dates_maxes.head()

,Record_Date,Record_Time,Modified_Date,value
0,2022-10-29,22:44:40,False,3
1,2022-11-01,22:09:07,False,3
2,2022-11-05,23:34:22,False,3
3,2022-11-15,22:52:41,False,5
4,2023-01-14,22:43:54,False,5


In [35]:
dedup_sat_dates = pd.concat([modified_dates_maxes,modified_dates_mins]).sort_values('Record_Date')

In [36]:
print(len(dedup_sat_dates))
dedup_sat_dates.head()

230


,Record_Date,Record_Time,Modified_Date,value
0,2022-10-28,02:09:04,True,4
0,2022-10-29,22:44:40,False,3
1,2022-10-31,00:06:00,True,4
1,2022-11-01,22:09:07,False,3
2,2022-11-04,01:48:01,True,2


In [37]:
# then add these to satisfaction data w count of dates < 2
sat_wout_duplicates.head()

,Record_Date,mood_type,value,Record_Datetime,Record_Time
0,2022-10-26,satisfaction,3,2022-10-26 20:55:38,20:55:38
1,2022-10-27,satisfaction,2,2022-10-27 22:19:43,22:19:43
2,2022-10-30,satisfaction,4,2022-10-30 22:08:04,22:08:04
3,2022-11-02,satisfaction,1,2022-11-02 22:06:19,22:06:19
4,2022-11-03,satisfaction,3,2022-11-03 22:05:15,22:05:15


In [38]:
# okay so all of this is unnecesary according to a genius I know
# just subtract 7 hrs from datetime instead of all of this!
# 7 is good because as of 7am the next day starts according to my Finch app,
# so the satisfaction score of the previous day must be submitted by then

In [39]:
satisfaction_data.head()

,mood_type,value,Record_Datetime,Record_Date
1,satisfaction,4,2024-02-11 04:06:37,2024-02-11
7,satisfaction,1,2022-11-02 22:06:19,2022-11-02
11,satisfaction,3,2024-11-03 01:17:46,2024-11-03
14,satisfaction,4,2022-12-14 03:26:55,2022-12-14
16,satisfaction,4,2023-05-20 00:25:57,2023-05-20


In [15]:
seven_hrs = timedelta(hours=7)
time_zone = timedelta(hours=3)
bc_start_date = datetime(2022,12,1)
bc_end_date = datetime(2022,12,31)

In [41]:
satisfaction_data = satisfaction_data.drop('Record_Date', axis=1)
satisfaction_data['Adjusted_Datetime'] = satisfaction_data['Record_Datetime'] - seven_hrs

# adjust for BC timezone
satisfaction_data['Adjusted_Datetime'] = satisfaction_data['Adjusted_Datetime'].map(lambda x: x-time_zone if (x > bc_start_date) & (x < bc_end_date) else x)

satisfaction_data['Record_Date'] = satisfaction_data['Adjusted_Datetime'].dt.date

In [42]:
satisfaction_data.head()

,mood_type,value,Record_Datetime,Adjusted_Datetime,Record_Date
1,satisfaction,4,2024-02-11 04:06:37,2024-02-10 21:06:37,2024-02-10
7,satisfaction,1,2022-11-02 22:06:19,2022-11-02 15:06:19,2022-11-02
11,satisfaction,3,2024-11-03 01:17:46,2024-11-02 18:17:46,2024-11-02
14,satisfaction,4,2022-12-14 03:26:55,2022-12-13 17:26:55,2022-12-13
16,satisfaction,4,2023-05-20 00:25:57,2023-05-19 17:25:57,2023-05-19


In [43]:
# check for duplicate dates
satisfaction_data.groupby('Record_Date').count().sort_values('value', ascending=False).head()

,mood_type,value,Record_Datetime,Adjusted_Datetime
Record_Date,,,,
2022-12-02,2,2,2,2
2024-12-29,2,2,2,2
2022-10-26,1,1,1,1
2024-05-02,1,1,1,1
2024-05-03,1,1,1,1


In [44]:
#hmm
bad_day1 = date(2022, 12, 2)
satisfaction_data[satisfaction_data['Record_Date'] == bad_day1]

,mood_type,value,Record_Datetime,Adjusted_Datetime,Record_Date
23,satisfaction,2,2022-12-03 01:05:45,2022-12-02 15:05:45,2022-12-02
2636,satisfaction,4,2022-12-02 10:40:24,2022-12-02 00:40:24,2022-12-02


In [45]:
bad_day2 = date(2024,12,29)
satisfaction_data[satisfaction_data['Record_Date'] == bad_day2]

,mood_type,value,Record_Datetime,Adjusted_Datetime,Record_Date
1501,satisfaction,3,2024-12-30 01:33:36,2024-12-29 18:33:36,2024-12-29
2622,satisfaction,3,2024-12-29 09:54:59,2024-12-29 02:54:59,2024-12-29


In [46]:
# ooh I think these 'bad days' are days that I was in BC, so + 3hrs to time for days I was in BC ?

In [ ]:
# conversely, I can remove these days as outliers, there are only a few out of 3000~ data records
# ooor manually adjust the record date 
# I believe the only reason these 2 days are off, despite the 3 hr shift and the 7 hr shift, 
#     is because I went and added the satisfaction score much later by going and changing it in 'history'

In [16]:
mood_data.head()

,mood_type,value,Record_Datetime,Record_Date
0,feeling,4,2023-05-06 14:00:58,2023-05-06
1,satisfaction,4,2024-02-11 04:06:37,2024-02-11
2,motivation,3,2022-11-02 09:04:11,2022-11-02
3,feeling,3,2023-07-12 19:02:37,2023-07-12
4,motivation,2,2025-03-01 13:39:47,2025-03-01


In [18]:
start_date = date(2025,3,29)
end_date = date(2025,4,11)

In [24]:
recent_df = mood_data[(mood_data['Record_Date'] > start_date) & (mood_data['Record_Date'] < end_date)].sort_values('Record_Datetime')
recent_df = recent_df.query("mood_type != 'motivation'")

In [27]:
recent_df.head(30)

,mood_type,value,Record_Datetime,Record_Date
2583,feeling,3,2025-03-30 06:53:26,2025-03-30
1824,feeling,3,2025-03-30 15:06:42,2025-03-30
2213,feeling,4,2025-03-30 17:17:48,2025-03-30
854,satisfaction,4,2025-03-30 23:57:18,2025-03-30
423,feeling,4,2025-03-31 02:18:57,2025-03-31
2922,feeling,4,2025-03-31 11:33:24,2025-03-31
2867,feeling,3,2025-03-31 12:43:09,2025-03-31
1179,feeling,4,2025-03-31 16:07:50,2025-03-31
3228,feeling,4,2025-03-31 17:16:54,2025-03-31
393,satisfaction,4,2025-03-31 23:13:35,2025-03-31
